# Tugas

1. Lakukan klasifikasi pada data MNIST dengan menggunakan model ANN

2. Anda diperbolehkan melakukan eksplorasi terhadap,
  * Metode pra pengolahan
  * Pemilihan fitur
  * Arsitektur ANN
  * Fungsi Aktiviasi

3. ANN diimplementasikan dengan menggunakan tensorflow.

4. DIKERJAKAN SECARA BERKELOMPOK

5. JELASKAN HASIL YANG ANDA DAPATKAN,
  * AKURASI
  * CONFUSION MATRIX
  * KONFIGURASI MODEL --> MULAI DARI PRA PENGOLAHAN SAMPAI ARSITEKTUR ANN

### Import Library

In [1]:
import cv2   # it will allow us to load our images into the script
import numpy as np   # used for reformatting our own images
import tensorflow as tf   # main library used to load data sets, build neural networks, train them, etc.
import matplotlib.pyplot as plt   # used for visualization
from collections import Counter

### Download dataset MNIST

In [2]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

c:\Users\bag4s\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


### Ekstrak Data

In [6]:
from skimage.feature import hog

# Extract HOG features from the images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        # Reshape the image to its original shape (28x28)
        image = image.reshape((28, 28))
        # Calculate HOG features
        hog_feature = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(hog_feature)
    return np.array(hog_features)

In [8]:
from sklearn.model_selection import train_test_split

# Extract features and labels
x, y = mnist.data.values.astype(int), mnist.target.astype(int)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train_hog = extract_hog_features(x_train)
x_test_hog = extract_hog_features(x_test)

### Normalize Data

In [9]:
x_train_hog = tf.keras.utils.normalize(x_train_hog, axis=1)
x_test_hog = tf.keras.utils.normalize(x_test_hog, axis=1)

### Preprocess Data

In [10]:
# Normalize the pixel values to the range [0, 1]
x_train_hog = x_train_hog / 255.0
x_test_hog = x_test_hog / 255.0

### Make model ANN

In [11]:
model = tf.keras.models.Sequential()

### Make layers

In [13]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))  # 10 classes for MNIST

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (462.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compiling Model

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_hog, y_train, epochs=25, batch_size=100, validation_data=(x_test, y_test))